In [1]:
import os
os.chdir('../..')

In [2]:
from modules.table_evaluator import load_data, TableEvaluator
# From https://github.com/Baukebrenninkmeijer/table-evaluator

from os import listdir
from os.path import isfile, join

import matplotlib.pyplot as plt
import numpy as np

import pandas as pd

# For the Python notebook
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [3]:
dataset = 'Chicago'
synth_dir = '../synth_data/' + dataset

files = ['../data/' + dataset + '/data.csv']
labels = ['original', 'WGAN_WI_NO', 'WGAN_WO_NO', 'SGAN_WI_OD', 'SGAN_WI_NO', 'WGAN_WI_OD', 'WGAN_WI_BO', 'WGAN_OR_OD', 'SGAN_WO_OD', 'SGAN_WO_NO']

for l in labels[1:]:
    files.append('../synth_data/' + dataset + '/' + l + '.csv')

In [4]:
if dataset is 'Chicago':
    continuous_columns = ["distance", "age", "departure_time"]
elif dataset is 'LPMC':
    continuous_columns = ['start_time_linear', 'age', 'distance', 'dur_walking', 'dur_cycling', 'dur_pt_access', 'dur_pt_rail', 'dur_pt_bus', 'dur_pt_int', 'dur_driving', 'cost_transit', 'cost_driving_fuel', 'driving_traffic_percent']

In [5]:
dfs = [pd.read_csv(file) for file in files]

In [6]:
cat_cols = list(set(dfs[0].columns) - set(continuous_columns))

In [7]:
n=7000

In [8]:
results = {}

print("random sampling:")
train = dfs[0].sample(int(len(dfs[0]) * 0.5))
test = dfs[0][~dfs[0].index.isin(train.index)]
table_evaluator = TableEvaluator(train.sample(int(n/2)), test.sample(int(n/2)), cat_cols, verbose=False)
res = table_evaluator.evaluate(target_col='choice')
results['sampling'] = res
print('--------------------------------------------------------')
print('')


for i, l in enumerate(labels[1:]):
    print(l + ":")
    train = dfs[0].sample(n)
    test = dfs[i+1].sample(n)
    test = test[train.columns]
    table_evaluator = TableEvaluator(train, test, cat_cols, verbose=False)
    res = table_evaluator.evaluate(target_col='choice')
    results[labels[i+1]] = res
    print('--------------------------------------------------------')
    print('')

random sampling:

Correlation metric: pearsonr

Classifier F1-scores and their Jaccard similarities:
                             f1_real  f1_fake  jaccard_similarity
index                                                            
DecisionTreeClassifier_fake   0.5926   0.6517              0.3937
DecisionTreeClassifier_real   0.6397   0.6143              0.4291
LogisticRegression_fake       0.7491   0.7514              0.8833
LogisticRegression_real       0.7437   0.7403              0.9280
MLPClassifier_fake            0.7529   0.7474              0.7786
MLPClassifier_real            0.7426   0.7486              0.7817
RandomForestClassifier_fake   0.7597   0.7669              0.7746
RandomForestClassifier_real   0.7454   0.7563              0.7978

Miscellaneous results:
                                         Result
Column Correlation Distance RMSE         0.0104
Column Correlation distance MAE          0.0066
Duplicate rows between sets (real/fake)  (0, 0)
nearest neighbor mean  


Miscellaneous results:
                                         Result
Column Correlation Distance RMSE         0.0211
Column Correlation distance MAE          0.0124
Duplicate rows between sets (real/fake)  (0, 0)
nearest neighbor mean                    1.5919
nearest neighbor std                     0.4785

Results:
                                                Result
Basic statistics                                1.0000
Correlation column correlations                 0.9902
Mean Correlation between fake and real columns  0.9103
1 - MAPE Estimator results                      0.9392
1 - MAPE 5 PCA components                       0.9162
Similarity Score                                0.9512
--------------------------------------------------------

WGAN_WI_BO:

Correlation metric: pearsonr

Classifier F1-scores and their Jaccard similarities:
                             f1_real  f1_fake  jaccard_similarity
index                                                            
Decisio

In [9]:
print("random sampling:")
train = dfs[0].sample(int(len(dfs[0]) * 0.5))
test = dfs[0][~dfs[0].index.isin(train.index)]
table_evaluator = TableEvaluator(train.sample(int(n/2)), test.sample(int(n/2)), cat_cols)
table_evaluator.evaluate(target_col='age', target_type='regr')
print('--------------------------------------------------------')
print('')

for i, l in enumerate(labels[1:]):
    print(l + ":")
    train = dfs[0].sample(n)
    test = dfs[i+1].sample(n)
    test = test[train.columns]
    table_evaluator = TableEvaluator(train, test, cat_cols)
    table_evaluator.evaluate(target_col='age', target_type='regr')
    print('--------------------------------------------------------')
    print('')

random sampling:

Correlation metric: pearsonr

Regressor MSE-scores and their Jaccard similarities:
                                   real    fake
fake_data_ElasticNet            16.1788 15.7971
fake_data_Lasso                 15.8025 15.5121
fake_data_RandomForestRegressor 11.1550 10.7124
fake_data_Ridge                 15.3451 15.1319
real_data_ElasticNet            16.0909 15.7190
real_data_Lasso                 15.6239 15.3465
real_data_RandomForestRegressor 10.6462 10.7873
real_data_Ridge                 15.2202 15.3199

Miscellaneous results:
                                         Result
Column Correlation Distance RMSE         0.0116
Column Correlation distance MAE          0.0072
Duplicate rows between sets (real/fake)  (0, 0)
nearest neighbor mean                    1.6625
nearest neighbor std                     0.4956

Results:
                                                Result
Basic statistics                                1.0000
Correlation column correlations    


Miscellaneous results:
                                         Result
Column Correlation Distance RMSE         0.0216
Column Correlation distance MAE          0.0126
Duplicate rows between sets (real/fake)  (0, 0)
nearest neighbor mean                    1.5847
nearest neighbor std                     0.4857

Results:
                                                Result
Basic statistics                                0.9930
Correlation column correlations                 0.9889
Mean Correlation between fake and real columns  0.8931
Correlation RMSE                                0.9760
1 - MAPE 5 PCA components                       0.9434
Similarity Score                                0.9589
--------------------------------------------------------

WGAN_OR_OD:

Correlation metric: pearsonr

Regressor MSE-scores and their Jaccard similarities:
                                   real    fake
fake_data_ElasticNet            16.2881 18.9923
fake_data_Lasso                 15.9071 18.

In [10]:
for cat in results['sampling'].keys():

    dct = {}
    for k in results.keys():
        dct[k] = results[k][cat]

    sorted_dct = {k: v for k, v in sorted(dct.items(), key=lambda item: item[1])[::-1]}

    print('Ranking based on "{}"'.format(cat))
    for i, item in enumerate(sorted_dct):
        print('  {:>2}. {:<25} - {:.3f}'.format(i+1, item, sorted_dct[item]))
        
    print()

Ranking based on "Basic statistics"
   1. WGAN_OR_OD                - 1.000
   2. WGAN_WI_OD                - 1.000
   3. SGAN_WI_NO                - 1.000
   4. SGAN_WI_OD                - 1.000
   5. sampling                  - 1.000
   6. WGAN_WI_BO                - 0.993
   7. WGAN_WI_NO                - 0.993
   8. SGAN_WO_OD                - 0.979
   9. WGAN_WO_NO                - 0.979
  10. SGAN_WO_NO                - 0.972

Ranking based on "Correlation column correlations"
   1. sampling                  - 0.997
   2. WGAN_WI_OD                - 0.990
   3. WGAN_WI_BO                - 0.989
   4. WGAN_WI_NO                - 0.986
   5. SGAN_WI_NO                - 0.985
   6. SGAN_WI_OD                - 0.985
   7. WGAN_OR_OD                - 0.977
   8. SGAN_WO_OD                - 0.975
   9. SGAN_WO_NO                - 0.974
  10. WGAN_WO_NO                - 0.964

Ranking based on "Mean Correlation between fake and real columns"
   1. WGAN_WI_NO                - 0.939
   2.

In [11]:
dct = {}
for k in results.keys():
    dct[k] = (results[k]['Basic statistics']+results[k]['Correlation column correlations']+results[k]['Mean Correlation between fake and real columns']+results[k]['1 - MAPE Estimator results'])/4

sorted_dct = {k: v for k, v in sorted(dct.items(), key=lambda item: item[1])[::-1]}

print('Ranking based on "Similarity Score wout PCA"')
for i, item in enumerate(sorted_dct):
    print('  {:>2}. {:<25} - {:.3f}'.format(i+1, item, sorted_dct[item]))

print()

Ranking based on "Similarity Score wout PCA"
   1. sampling                  - 0.974
   2. WGAN_WI_NO                - 0.971
   3. SGAN_WI_NO                - 0.967
   4. WGAN_OR_OD                - 0.962
   5. WGAN_WI_OD                - 0.960
   6. WGAN_WI_BO                - 0.955
   7. SGAN_WO_NO                - 0.954
   8. SGAN_WI_OD                - 0.954
   9. SGAN_WO_OD                - 0.949
  10. WGAN_WO_NO                - 0.948

